## Import libraries

In [34]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [35]:
train_data = np.load('dataset/Shoes - Female - Train.npz')
validation_data = np.load('dataset/Shoes - Female - Validation.npz')
test_data = np.load('dataset/Shoes - Female - Test.npz')

In [36]:
train_data.files

['images', 'labels']

In [37]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [38]:
images_train.shape

(2886, 120, 90, 3)

In [39]:
labels_train.shape

(2886,)

In [40]:
# Data augmentation
# Flip the image by flip the axis 2
# we have 4 axis now in images (no. of pic, y, x, channel)
# so, if we want to flip horizontally, we flip axis 2

flipped_images_train = np.flip(images_train, axis = 2)
flipped_images_val = np.flip(images_val, axis = 2)
flipped_images_test = np.flip(images_test, axis = 2)

In [41]:
# Don't forget to concatenate labels with itself, cuz we increase pic in images, so labels have to be increased too.
images_train = np.concatenate((images_train, flipped_images_train))
labels_train = np.concatenate((labels_train, labels_train))

images_val = np.concatenate((images_val, flipped_images_val))
labels_val = np.concatenate((labels_val, labels_val))

images_test = np.concatenate((images_test, flipped_images_test))
labels_test = np.concatenate((labels_test, labels_test))

In [42]:
images_val.shape

(720, 120, 90, 3)

In [43]:
labels_val.shape

(720,)

In [44]:
images_test.shape

(720, 120, 90, 3)

In [45]:
labels_test.shape

(720,)

In [46]:
np.unique(labels_train)

array([0, 1, 2, 3, 4, 5])

In [47]:
# Standardize the data
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Model

### Define hyperparameters

In [48]:
EPOCHS = 15
BATCH_SIZE = 64

In [49]:
HP_KERNEL_SIZE = hp.HParam('kernel size', hp.Discrete([3,5,7]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([64,128]))
HP_LAMBDA_REG = hp.HParam('lambda reg', hp.Discrete([1e-5,5e-5,1e-4]))

In [50]:
METRIC = 'accuracy'

with tf.summary.create_file_writer(r'logs/Model 1 (Shoes_Female_L2)/hparam_tuning').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE, HP_KERNEL_NUM, HP_LAMBDA_REG],
        metrics = [hp.Metric(METRIC, display_name = 'accuracy')]
    )

### Model architecture

In [51]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', input_shape = (120,90,3), kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation = 'relu', kernel_regularizer = tf.keras.regularizers.L2(hparams[HP_LAMBDA_REG])),
        tf.keras.layers.Dense(6, activation = 'softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(name = 'sparse_categorical_crossentropy')
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy','sparse_categorical_crossentropy'])
    
    log_dir = "Logs\\Model 1 (Shoes_Female_L2)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Boots','Ballerina', 'Trainers/Sneakers',
                                                        'High heels', 'Sandals/Flip flops/Slippers',
                                                        'Others'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_sparse_categorical_crossentropy', patience = 2, restore_best_weights = True)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1, profile_batch = 0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy, _ = model.evaluate(images_val, labels_val)
    
    model.save(r'saved_models\Model 1 (Shoes_Female_L2)\Run-{}'.format(session_num))
    
    return accuracy

In [52]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [53]:
session_num = 1
for kernel_size in HP_KERNEL_SIZE.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        for lambda_reg in HP_LAMBDA_REG.domain.values:
            hparams = {
                HP_KERNEL_SIZE : kernel_size,
                HP_KERNEL_NUM : kernel_num,
                HP_LAMBDA_REG : lambda_reg
            }
            run_name = f'run-{session_num}'
            print('--- Starting Trial:',run_name)
            print({h.name: hparams[h] for h in hparams})
            run("Logs/Model 1 (Shoes_Female_L2)/hparam_tuning/" + run_name, hparams, session_num)

            session_num += 1

--- Starting Trial: run-1
{'kernel size': 3, 'kernel num': 64, 'lambda reg': 1e-05}
Epoch 1/15
91/91 [==============================] - 58s 634ms/step - loss: 1.1441 - accuracy: 0.6157 - sparse_categorical_crossentropy: 1.1372 - val_loss: 0.6877 - val_accuracy: 0.7861 - val_sparse_categorical_crossentropy: 0.6802
Epoch 2/15
91/91 [==============================] - 58s 634ms/step - loss: 0.5933 - accuracy: 0.7949 - sparse_categorical_crossentropy: 0.5852 - val_loss: 0.5891 - val_accuracy: 0.7972 - val_sparse_categorical_crossentropy: 0.5803
Epoch 3/15
91/91 [==============================] - 58s 635ms/step - loss: 0.4400 - accuracy: 0.8540 - sparse_categorical_crossentropy: 0.4302 - val_loss: 0.4705 - val_accuracy: 0.8681 - val_sparse_categorical_crossentropy: 0.4600
Epoch 4/15
91/91 [==============================] - 58s 636ms/step - loss: 0.3213 - accuracy: 0.8928 - sparse_categorical_crossentropy: 0.3101 - val_loss: 0.4313 - val_accuracy: 0.8861 - val_sparse_categorical_crossentropy:

Epoch 4/15
91/91 [==============================] - 135s 1s/step - loss: 0.3572 - accuracy: 0.8786 - sparse_categorical_crossentropy: 0.3470 - val_loss: 0.5563 - val_accuracy: 0.8542 - val_sparse_categorical_crossentropy: 0.5454
Epoch 5/15
23/23 [==============================] - 5s 197ms/step - loss: 0.4611 - accuracy: 0.8625 - sparse_categorical_crossentropy: 0.4513
INFO:tensorflow:Assets written to: saved_models\Model 1 (Shoes_Female_L2)\Run-4\assets
--- Starting Trial: run-5
{'kernel size': 3, 'kernel num': 128, 'lambda reg': 5e-05}
Epoch 1/15
91/91 [==============================] - 140s 2s/step - loss: 1.4443 - accuracy: 0.5324 - sparse_categorical_crossentropy: 1.4124 - val_loss: 0.7741 - val_accuracy: 0.7431 - val_sparse_categorical_crossentropy: 0.7442
Epoch 2/15
91/91 [==============================] - 138s 2s/step - loss: 0.6939 - accuracy: 0.7696 - sparse_categorical_crossentropy: 0.6647 - val_loss: 0.5653 - val_accuracy: 0.8319 - val_sparse_categorical_crossentropy: 0.5367

91/91 [==============================] - 89s 977ms/step - loss: 0.3636 - accuracy: 0.8889 - sparse_categorical_crossentropy: 0.3287 - val_loss: 0.4887 - val_accuracy: 0.8417 - val_sparse_categorical_crossentropy: 0.4521
Epoch 5/15
91/91 [==============================] - 90s 991ms/step - loss: 0.3127 - accuracy: 0.9058 - sparse_categorical_crossentropy: 0.2749 - val_loss: 0.5031 - val_accuracy: 0.8528 - val_sparse_categorical_crossentropy: 0.4638
Epoch 6/15
91/91 [==============================] - 89s 983ms/step - loss: 0.2327 - accuracy: 0.9319 - sparse_categorical_crossentropy: 0.1923 - val_loss: 0.4701 - val_accuracy: 0.8639 - val_sparse_categorical_crossentropy: 0.4289
Epoch 7/15
91/91 [==============================] - 90s 989ms/step - loss: 0.2025 - accuracy: 0.9446 - sparse_categorical_crossentropy: 0.1600 - val_loss: 0.4361 - val_accuracy: 0.8819 - val_sparse_categorical_crossentropy: 0.3927
Epoch 8/15
91/91 [==============================] - 92s 1s/step - loss: 0.1792 - accura

--- Starting Trial: run-12
{'kernel size': 5, 'kernel num': 128, 'lambda reg': 0.0001}
Epoch 1/15
91/91 [==============================] - 218s 2s/step - loss: 1.5281 - accuracy: 0.5100 - sparse_categorical_crossentropy: 1.4711 - val_loss: 0.8802 - val_accuracy: 0.7361 - val_sparse_categorical_crossentropy: 0.8278
Epoch 2/15
91/91 [==============================] - 218s 2s/step - loss: 0.8080 - accuracy: 0.7370 - sparse_categorical_crossentropy: 0.7587 - val_loss: 0.6922 - val_accuracy: 0.7819 - val_sparse_categorical_crossentropy: 0.6441
Epoch 3/15
91/91 [==============================] - 224s 2s/step - loss: 0.6435 - accuracy: 0.7975 - sparse_categorical_crossentropy: 0.5952 - val_loss: 0.5559 - val_accuracy: 0.8403 - val_sparse_categorical_crossentropy: 0.5079
Epoch 4/15
91/91 [==============================] - 237s 3s/step - loss: 0.4930 - accuracy: 0.8484 - sparse_categorical_crossentropy: 0.4451 - val_loss: 0.6106 - val_accuracy: 0.8028 - val_sparse_categorical_crossentropy: 0.56

91/91 [==============================] - 153s 2s/step - loss: 0.2861 - accuracy: 0.9163 - sparse_categorical_crossentropy: 0.2310 - val_loss: 0.5265 - val_accuracy: 0.8653 - val_sparse_categorical_crossentropy: 0.4714
Epoch 10/15
23/23 [==============================] - 5s 200ms/step - loss: 0.4745 - accuracy: 0.8792 - sparse_categorical_crossentropy: 0.4197
INFO:tensorflow:Assets written to: saved_models\Model 1 (Shoes_Female_L2)\Run-15\assets
--- Starting Trial: run-16
{'kernel size': 7, 'kernel num': 128, 'lambda reg': 1e-05}
Epoch 1/15
91/91 [==============================] - 384s 4s/step - loss: 1.7934 - accuracy: 0.3735 - sparse_categorical_crossentropy: 1.7843 - val_loss: 1.7838 - val_accuracy: 0.1833 - val_sparse_categorical_crossentropy: 1.7733
Epoch 2/15
91/91 [==============================] - 380s 4s/step - loss: 1.7946 - accuracy: 0.3352 - sparse_categorical_crossentropy: 1.7835 - val_loss: 1.5489 - val_accuracy: 0.4444 - val_sparse_categorical_crossentropy: 1.5364
Epoch 3